In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import cv2, os
import numpy as np

In [2]:
import keras
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/aakashjuseja/anaconda3/envs/first-project/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
size=256
classes=5
image_dir='train_images'
mask_dir='train_masks'
data_shape = size*size


In [4]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
import random
percent=0.80
val_image_dir='val_images'
val_mask_dir='val_masks'
data_shape=size*size
import os


def onehot_label(label):
    """
    Converts labels to onehot array.
    Returns: (h, w, classes)
    """
    classes = classes
    mask = np.eye(classes)[label]
    return mask
def hot_encoder(label):
    mask = np.zeros((size, size, classes))
    for c in range(classes):
        mask[:,:,c] = (label==c).astype(int)
        
    return mask
    
def preprocess_inputs(self, X):
    return imagenet_utils.preprocess_input(X)

def reshape_labels(y):
    return np.reshape(y, (data_shape, classes))
    

def data_gen(img_folder, mask_folder, batch_size):
  c = 0
  n = os.listdir(img_folder) #List of training images
  random.shuffle(n)
  
  while (True):
    img = np.zeros((batch_size,size , size, 3)).astype('float')
    mask = np.zeros((batch_size, size*size, 5)).astype('float')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img = cv2.imread(img_folder+'/'+n[i])/255.0
      #train_img =  cv2.resize(train_img, (size, size))# Read an image from folder and resize

      img[i-c] = train_img #add to array - img[0], img[1], and so on.
    
      #train_mask = cv2.imread(mask_folder+'/'+n[i], 0)

      train_mask = hot_encoder(cv2.resize(cv2.imread(mask_folder+'/'+n[i], 0),(size,size)))
      #train_mask = train_mask.reshape(512, 512, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
      train_mask=np.reshape(train_mask, (size*size, classes))
      mask[i-c] = train_mask

    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      random.shuffle(n)
                  # print "randomizing again"
    yield img, mask


In [5]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation, Reshape, Permute
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Conv2DTranspose,concatenate,Dropout
from keras.layers import BatchNormalization


In [6]:
kernel = 3
input_shape= (size, size, 3)
classes=5

img_input = Input(shape=(input_shape))
X = img_input

# Encoding layers
# Block 1 
X = Convolution2D(64, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(64, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = MaxPooling2D()(X)

# Block 2
X = Convolution2D(128, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(128, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = MaxPooling2D()(X)

# Block 3
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = MaxPooling2D()(X)

# Block 4
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = MaxPooling2D()(X)

# Block 5
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = MaxPooling2D()(X)

# Decoding layers
# Block 1
X = UpSampling2D()(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)

# Block 2
X = UpSampling2D()(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(512, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)

# Block 3
X = UpSampling2D()(X)
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(256, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(128, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)

# Block 4
X = UpSampling2D()(X)
X = Convolution2D(128, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(64, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)

# Block 5
X = UpSampling2D()(X)
X = Convolution2D(64, (kernel, kernel), padding="same")(X)
X = BatchNormalization()(X)
X = Activation("relu")(X)
X = Convolution2D(classes, (1, 1), padding="valid")(X)
X = BatchNormalization()(X)
X = Reshape((input_shape[0]*input_shape[1], classes), input_shape = input_shape)(X)
X = Permute((1, 2))(X)
X = Activation("softmax")(X)
model = Model(img_input, X)

In [11]:
batch_size=12
train_gen = data_gen(image_dir,mask_dir, batch_size = batch_size)
val_gen = data_gen(val_image_dir,val_mask_dir, batch_size =batch_size)

In [8]:
from keras import optimizers
opt=optimizers.Adam(lr=0.0001)

In [9]:
#from keras.models import load_model
#model=load_model('SegNet_C_001-78-0.74.h5')

In [13]:

model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=['accuracy'])
num_training_samples=np.array(os.listdir(image_dir)).shape[0]
num_epochs=150
num_validation_samples=np.array(os.listdir(val_image_dir)).shape[0]


    # load data
#(num_training_samples // batch_size)
filepath = "SegNet_001-{epoch:02d}-{val_loss:.2f}.h5"
tb_cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True)

model.fit_generator(generator=train_gen,
                                      steps_per_epoch=(num_training_samples // batch_size),
                                      epochs=num_epochs,
                                      verbose=1,
                                        validation_data=val_gen,
                                        validation_steps=(num_validation_samples // batch_size),
                                      use_multiprocessing=True,
                                      workers=1,
                                      max_queue_size=1,
                                       callbacks=[tb_cb],
                                       initial_epoch=5)

Epoch 6/150
611/611 [==============================] - 528s 864ms/step - loss: 1.0592 - acc: 0.5979 - val_loss: 1.0060 - val_acc: 0.6259
Epoch 7/150
611/611 [==============================] - 534s 874ms/step - loss: 0.8711 - acc: 0.6861 - val_loss: 0.9390 - val_acc: 0.6123
Epoch 8/150
611/611 [==============================] - 528s 864ms/step - loss: 0.8029 - acc: 0.7144 - val_loss: 0.7962 - val_acc: 0.7011
Epoch 9/150
611/611 [==============================] - 523s 856ms/step - loss: 0.7665 - acc: 0.7300 - val_loss: 0.8856 - val_acc: 0.6611
Epoch 10/150
611/611 [==============================] - 524s 857ms/step - loss: 0.7222 - acc: 0.7446 - val_loss: 0.8078 - val_acc: 0.7108
Epoch 11/150
611/611 [==============================] - 521s 853ms/step - loss: 0.6949 - acc: 0.7553 - val_loss: 0.8019 - val_acc: 0.7085
Epoch 12/150
611/611 [==============================] - 521s 852ms/step - loss: 0.6751 - acc: 0.7625 - val_loss: 0.7288 - val_acc: 0.7353
Epoch 13/150
611/611 [================

KeyboardInterrupt: 